In [ ]:
import gdata.photos.service
import gdata.media
import gdata.geo

import os

import oauth2client

import httplib2

from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage

import threading

import datetime

# This part is required to come up with the pop-up with which Jawbone will ask for permissions
from IPython.display import display
from IPython.display import Javascript as JS

import requests
import os

def OAuth2Login(client_secrets, credential_store, email):
    scope='https://picasaweb.google.com/data/'
    user_agent='PicasaMigrator'

    storage = Storage(credential_store)
    credentials = storage.get()
    if credentials is None or credentials.invalid:
        flow = flow_from_clientsecrets(client_secrets, scope=scope, redirect_uri='urn:ietf:wg:oauth:2.0:oob')
        uri = flow.step1_get_authorize_url()
        
        threading.Timer(1, lambda: display(JS("window.open('%s')" % uri))).start()

        code = raw_input('Enter the authentication code: ').strip()
        credentials = flow.step2_exchange(code)
        storage.put(credentials)

    if (credentials.token_expiry - datetime.datetime.utcnow()) < datetime.timedelta(minutes=5):
        http = httplib2.Http()
        http = credentials.authorize(http)
        credentials.refresh(http)

    gd_client = gdata.photos.service.PhotosService(source=user_agent,
                                               email=email,
                                               additional_headers={'Authorization' : 'Bearer %s' % credentials.access_token})

    return gd_client


# Downloads all the photos in the album ALBUM_ID to the specified DIRECTORY, with full resolution
def download_album(gd_client, album_id, directory):
    
    if not os.path.exists(directory):
        os.makedirs(directory)
   
    # The imgmax=d indicates that we want the original photos
    photos = gd_client.GetFeed('/data/feed/api/user/default/albumid/%s?&imgmax=d' % (album_id))
    
    count = 1;
    for photo in photos.entry:
        download_file(photo, directory, str(count))
        count += 1

# Returns the proper URL for the media file (e.g. the highest resolution video for a video file)
def get_entry(photo):
    ""
    
    result = photo.media.content[0]
    for entry in photo.media.content:
        if entry.medium=="video" and result.medium=="video":
            if int(result.width)<int(entry.width):
                result = entry
        if entry.medium=="video" and result.medium=="image":
            result = entry
            
    return result


def download_file(photo, directory, index):
    "Download the data at URL to the current directory."
    #print "============", photo.gphoto_id.text, "==============="
    
    entry = get_entry(photo)
    url = entry.url
    
    extension = entry.type[entry.type.rindex('/') + 1:]
    basename = index + '.' + extension
    
    metadata = os.path.join(directory,index+".xml")
    #print metadata
    with open(metadata,'wb') as f:
        f.write(entry.ToString())
    
    filename = os.path.join(directory,basename)
    #print filename
    if os.path.exists(filename):
        print "+",
        #print "Cached: %s" % (basename,)
        return
    
    
    #print "Downloading %s" % (basename)
    #print url
    print ".",
    r = requests.get(url)

    with open(filename,'wb') as f:
        f.write(r.content)

    
email = 'gregory@ipeirotis.com'
configdir = os.path.expanduser('/scratch/picasa')
client_secrets = os.path.join(configdir, 'client_secret.json')
credential_store = os.path.join(configdir, 'credentials.dat')
gd_client_source = OAuth2Login(client_secrets, credential_store, email)

albums = gd_client_source.GetUserFeed()

cnt=1
s = len(albums.entry)
for album in albums.entry:
    
    
    album_title = album.title.text
    directory = os.path.join(configdir,  album_title + ' ' + album.gphoto_id.text)
    
    print '\n', cnt, '/', s, '. Album: %s (%s) [%s]' % (album_title, album.numphotos.text,  album.gphoto_id.text)
    cnt = cnt+1

    download_album(gd_client_source, album.gphoto_id.text, directory)


In [9]:
xmlfiles = !find /scratch/picasa -type f -name '*.xml'

In [10]:
len(xmlfiles)

5541

In [14]:
from lxml import etree


types = set()
for f in xmlfiles:

    # Read the file
    filecontent = open(f, "r").read()
    #print filecontent
    
    doc  = etree.XML(filecontent)
    t = doc.xpath("/*[local-name()='content']")[0].get("type")
    types.add(t)
    
print types

set(['image/gif', 'video/mpeg4', 'image/jpeg'])
